# Cleaning the data

Transforming the data from a mongodb collection to a csv file 


In [1]:
import pymongo

# Connection to Mongo DB
try:
    conn=pymongo.MongoClient()
    print("connected")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e )

db = conn["idealista"]
collection = db["house_data"]
urls = db["urls"]


connected


In [19]:
import pandas as pd
df = pd.DataFrame()

In [23]:
# Exploring the data

collection.find_one()

{'_id': ObjectId('5dece54c833639ed33008205'),
 'price': '173.000 €',
 'features': [' 35 m² ', ' 2 hab. '],
 'description': ' "En la zona peatonal del barrio de Sant Pere i Santa Caterina encontramos esta propiedad en la segunda planta del edificio antiguo, sin ascensor, típico de la zona. Se trata de un estudio reformado hace unos años con calidades apropiadas para las exigencias de la zona. El piso se vende amueblado tal y como está en las fotos. Puede ser interesante para su alquiler (alrededor de 850€/mes). Ubicado en pleno centro de Barcelona, con todos los servicios cerca, sin necesidad de tener el coche.Ubicado en la mejor y más buscada área del Born, esta obra se encuentra a pocos pasos de Arc de Triomf, Parc de la Ciutadella, muy bien comunicada con transporte público (tren, metro, autobuses)." ',
 'details_house': ['35 m² construidos',
  '2 habitaciones',
  '1 baño',
  'Construido en 1848'],
 'details_building': [' Aire acondicionado '],
 'energy_class': 'd',
 'address': [' ',

In [20]:
# checking how to conver price to integer

cursor = collection.find({},{"price":1})
l = []
for doc in cursor:
    price = doc["price"]
    price = price.replace("€","").replace(".","")
    price = price.strip()
    try:
        price = int(price)
    except:
        print(price)
        price = 0
    l.append(price)

df["price"]=l
print("done")
    

done


In [60]:
# features
import math
import numpy as np
cursor = collection.find({},{"features":1})
f =set()
m2 = []
ascensor = []
exterior = []
planta = []
garaje = []
habs = []

for doc in cursor:
    ascensor.append(0)
    exterior.append(0)
    garaje.append(0)
    planta.append(0)
    habs.append(0)
    last_idx = len(ascensor)-1
    for feature in doc["features"]:        
        if "m²" in feature:
            m = feature.strip()
            m = feature.split(" ")[1] 
            m = m.strip()
            m = int(math.floor(float(m)))
            m2.append(m)
            continue
        if "con ascensor" in feature:
            ascensor[last_idx] = 1
        if "exterior" in feature:
            exterior[last_idx]=1
        if "ª planta" in feature:
            p = feature.split("ª planta")
            p = int(p[0])
            planta[last_idx] = p
            continue        
        if "Garaje incluido" in feature:
            garaje[last_idx] = 1
            continue
        if "hab" in feature:
            h = feature.strip()
            h = feature.split(" ")[1]
            h = int(h)
            habs[last_idx] = h
            continue
        try:
            for skip in [ "Bajo","Entreplanta","exterior","interior","ascensor","Garaje"]:
                if skip in feature:
                    raise Exception("skip")
        except:
            continue
        f.add(feature) # untread feature
#df["price"]=l
f
    

{'opc.'}

In [64]:
len(m2),len(planta),len(ascensor),len(exterior),len(garaje),len(habs)

(13937, 13937, 13937, 13937, 13937, 13937)

In [46]:
m2

[35,
 55,
 56,
 49,
 60,
 55,
 45,
 59,
 73,
 55,
 80,
 49,
 70,
 68,
 60,
 50,
 72,
 66,
 76,
 65,
 61,
 65,
 55,
 32,
 42,
 50,
 65,
 50,
 67,
 34,
 78,
 62,
 66,
 110,
 75,
 53,
 42,
 50,
 64,
 70,
 70,
 97,
 66,
 73,
 55,
 41,
 36,
 72,
 53,
 35,
 81,
 49,
 100,
 40,
 70,
 65,
 84,
 66,
 78,
 55,
 48,
 92,
 55,
 69,
 55,
 75,
 70,
 70,
 92,
 64,
 40,
 60,
 53,
 50,
 46,
 70,
 61,
 68,
 72,
 72,
 55,
 65,
 63,
 95,
 62,
 65,
 65,
 65,
 35,
 69,
 85,
 58,
 80,
 46,
 40,
 51,
 58,
 60,
 54,
 57,
 64,
 70,
 70,
 70,
 60,
 83,
 67,
 50,
 49,
 53,
 65,
 83,
 37,
 57,
 75,
 90,
 43,
 85,
 95,
 54,
 76,
 60,
 61,
 76,
 63,
 65,
 46,
 60,
 64,
 42,
 50,
 80,
 64,
 95,
 40,
 60,
 65,
 93,
 35,
 66,
 60,
 94,
 33,
 75,
 59,
 47,
 70,
 55,
 47,
 50,
 35,
 70,
 42,
 75,
 70,
 45,
 52,
 96,
 89,
 76,
 100,
 69,
 80,
 54,
 65,
 85,
 53,
 66,
 46,
 57,
 43,
 75,
 75,
 65,
 50,
 72,
 100,
 73,
 45,
 54,
 70,
 27,
 75,
 55,
 67,
 70,
 65,
 69,
 65,
 48,
 60,
 52,
 41,
 62,
 80,
 42,
 79,
 67,
 64,
 

In [15]:
df

,a,b
0,10,1
1,20,2
2,30,3
